In [1]:
import torch
import numpy as np

from datasets.dataset import transform_dataset, kfold_dataset
from R2Ntab import train as train_r2ntab, R2Ntab
from DRNet import train as train_drnet, DRNet

import sys

import matplotlib.pyplot as plt

In [2]:
# Read datasets
name = 'heloc'
X, Y, X_headers, Y_headers = transform_dataset(name, method='onehot-compare', negations=False, labels='binary')
datasets = kfold_dataset(X, Y, shuffle=1)
X_train, X_test, Y_train, Y_test = datasets[0]
train_set = torch.utils.data.TensorDataset(torch.Tensor(X_train.to_numpy()), torch.Tensor(Y_train))
test_set = torch.utils.data.TensorDataset(torch.Tensor(X_test.to_numpy()), torch.Tensor(Y_test))

In [3]:
# Train R2N-tab
# Default learning rate (1e-2), and_lam (1e-2), and and_lam (1e-5) usually work the best. A large epochs number is necessary for a sparse rule set i.e 10000 epochs.
net = R2Ntab(train_set[:][0].size(1), 50, 1)
train_r2ntab(net, train_set, test_set=test_set, device='cpu', lr_rules=1e-2, lr_cancel=5e-3, epochs=1500, batch_size=400,
      and_lam=1e-2, or_lam=1e-5, cancel_lam=1e-3, num_alter=500, track_performance=True)

Epoch: 100%|██████████| 1500/1500 [07:08<00:00,  3.50it/s, rules cancelled=73, loss=0.638, epoch accu=0.625, test accu=0.71, num rules=9, sparsity=0.867]  


([0.5219885110855103,
  0.5219885110855103,
  0.5219885110855103,
  0.5219885110855103,
  0.5219885110855103,
  0.5219885110855103,
  0.5219885110855103,
  0.5219885110855103,
  0.5219885110855103,
  0.5219885110855103,
  0.5219885110855103,
  0.5219885110855103,
  0.5219885110855103,
  0.5219885110855103,
  0.5219885110855103,
  0.5219885110855103,
  0.5219885110855103,
  0.5219885110855103,
  0.5219885110855103,
  0.5219885110855103,
  0.5219885110855103,
  0.5219885110855103,
  0.5219885110855103,
  0.5219885110855103,
  0.5219885110855103,
  0.5344168543815613,
  0.52390056848526,
  0.5224665403366089,
  0.5243785977363586,
  0.5487571954727173,
  0.5219885110855103,
  0.538240909576416,
  0.5315487384796143,
  0.5482791662216187,
  0.5377628803253174,
  0.5683556199073792,
  0.5497131943702698,
  0.6018164157867432,
  0.6515296101570129,
  0.6787762641906738,
  0.6677820086479187,
  0.6696940660476685,
  0.6701720952987671,
  0.6787762641906738,
  0.6945506930351257,
  0.688336491

In [4]:
# Get accuracy and the rule net
accu = (net.predict(np.array(X_test)) == Y_test).mean()
rules = net.get_rules(X_headers)
print(f'Accuracy: {accu}, num rules: {len(rules)}, num conditions: {sum(map(len, rules))}')

Accuracy: 0.7103250478011472, num rules: 9, num conditions: 122


In [ ]:
print(rules)

In [ ]:
plt.plot(performance)
plt.ylabel("test accuracy")
plt.xlabel("epoch")
plt.show()

In [ ]:
plt.plot(loss_ot)
plt.ylabel("loss")
plt.xlabel("epoch")
plt.show()

In [ ]:
plt.plot(cancel_sparsity)
plt.plot(rule_sparsity)
plt.ylabel('sparsity')
plt.xlabel('epoch')
plt.legend(['cancel sparsity', 'rule sparsity'])
plt.show()